In [1]:
import pandas as pd

In [2]:
column_names = ['account_id', 
                'name', 
                'point', 
                'call_count', 
                'total_call_time', 
                'total_exclusive_time', 
                'min_call_time', 
                'max_call_time', 
                'sum_of_squares', 
                'instances', 
                'language', 
                'app_name', 
                'app_id', 
                'scope', 
                'host', 
                'display_host', 
                'pid', 
                'agent_version', 
                'labels']

In [ ]:
df = pd.read_csv(r'/kaggle/input/rl-hack-data/Кейс 1/data.tsv', sep='\t', header=None, names=column_names)
df.head()

In [22]:
filtered_df = df[
    (df['language'] == 'java') &
    (df['app_name'] == '[GMonit] Collector') &
    (df['scope'].isna()) &
    (df['name'] == 'HttpDispatcher')
]

grouped_df = filtered_df.groupby('point').agg({
    'total_call_time': 'sum',
    'call_count': 'sum'
}).reset_index()

grouped_df['average_call_time'] = grouped_df['total_call_time'] / grouped_df['call_count']

sorted_df = grouped_df.sort_values(by='point')

result_df = sorted_df[['point', 'average_call_time']]

In [24]:
result_df

,point,average_call_time
0,2024-04-15 23:32:00,0.007722
1,2024-04-15 23:33:00,0.008363
2,2024-04-15 23:34:00,0.008709
3,2024-04-15 23:35:00,0.008286
4,2024-04-15 23:36:00,0.010874
...,...,...
43267,2024-05-16 00:54:00,0.009245
43268,2024-05-16 00:55:00,0.010571
43269,2024-05-16 00:56:00,0.011785
43270,2024-05-16 00:57:00,0.010940


In [26]:
result_df.rename(columns={'point': 'timestamp', 'average_call_time': 'value'}, inplace=True)

/tmp/ipykernel_34/3839128190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.rename(columns={'point': 'timestamp', 'average_call_time': 'value'}, inplace=True)


In [28]:
result_df.to_csv('web_response.csv')

In [32]:
filtered_df = df[
    (df['language'] == 'java') &
    (df['app_name'] == '[GMonit] Collector') &
    (df['scope'].isna()) &
    (df['name'] == 'HttpDispatcher')
]

grouped_df = filtered_df.groupby('point').agg({
    'call_count': 'sum'
}).reset_index()

sorted_df = grouped_df.sort_values(by='point')

In [33]:
sorted_df

,point,call_count
0,2024-04-15 23:32:00,3898.0
1,2024-04-15 23:33:00,3917.5
2,2024-04-15 23:34:00,3993.0
3,2024-04-15 23:35:00,3991.5
4,2024-04-15 23:36:00,3915.5
...,...,...
43267,2024-05-16 00:54:00,2560.0
43268,2024-05-16 00:55:00,2491.0
43269,2024-05-16 00:56:00,2429.0
43270,2024-05-16 00:57:00,2373.5


In [34]:
sorted_df.rename(columns={'point': 'timestamp', 'call_count': 'value'}, inplace=True)

In [35]:
sorted_df.to_csv('throughput.csv')

In [36]:
filtered_df = df[
    (df['language'] == 'java') &
    (df['app_name'] == '[GMonit] Collector') &
    (df['scope'].isna()) &
    (df['name'] == 'Apdex')
]

agg_df = filtered_df.groupby('point').agg(
    s=pd.NamedAgg(column='call_count', aggfunc='sum'),
    t=pd.NamedAgg(column='total_call_time', aggfunc='sum'),
    f=pd.NamedAgg(column='total_exclusive_time', aggfunc='sum')
).reset_index()

agg_df.fillna(0, inplace=True)

agg_df[' '] = (agg_df['s'] + agg_df['t'] / 2) / (agg_df['s'] + agg_df['t'] + agg_df['f'])

agg_df.sort_values(by='point', inplace=True)

In [37]:
agg_df

,point,s,t,f,
0,2024-04-15 23:32:00,3896.0,1.5,0.5,0.999679
1,2024-04-15 23:33:00,3916.5,0.0,1.0,0.999745
2,2024-04-15 23:34:00,3991.5,0.5,1.0,0.999687
3,2024-04-15 23:35:00,3990.5,0.5,0.5,0.999812
4,2024-04-15 23:36:00,3912.5,2.0,1.0,0.999489
...,...,...,...,...,...
43267,2024-05-16 00:54:00,2555.0,4.0,1.0,0.998828
43268,2024-05-16 00:55:00,2485.0,4.5,1.5,0.998495
43269,2024-05-16 00:56:00,2420.0,7.0,2.0,0.997736
43270,2024-05-16 00:57:00,2366.0,5.5,2.0,0.997999


In [38]:
agg_df.rename(columns={'point': 'timestamp', ' ': 'value'}, inplace=True)

In [40]:
agg_df = agg_df.drop(columns=['s', 't', 'f'], axis=1)

In [41]:
agg_df.to_csv('apdex.csv')

In [45]:
filtered_df = df[(df['language'] == 'java') & 
                 (df['app_name'] == '[GMonit] Collector') & 
                 (df['scope'].isna()) & 
                 (df['name'].isin(['HttpDispatcher', 'Errors/allWeb']))]

grouped = filtered_df.groupby(['point', 'name'])['call_count'].sum().unstack(fill_value=0)

grouped['result'] = grouped['Errors/allWeb'] / grouped['HttpDispatcher']

result = grouped['result'].reset_index().sort_values(by='point')

In [46]:
result

,point,result
0,2024-04-15 23:32:00,0.000128
1,2024-04-15 23:33:00,0.000128
2,2024-04-15 23:34:00,0.000000
3,2024-04-15 23:35:00,0.000000
4,2024-04-15 23:36:00,0.000000
...,...,...
43267,2024-05-16 00:54:00,0.000000
43268,2024-05-16 00:55:00,0.000000
43269,2024-05-16 00:56:00,0.000206
43270,2024-05-16 00:57:00,0.000211


In [47]:
result.rename(columns={'point': 'timestamp', 'result': 'value'}, inplace=True)

In [48]:
result.to_csv('error.csv')